Using Bing to gound an LLM
Option 1: Grounding with Bing Search Resource https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=code-example
For the Bing Grounding, you wnat to make sure you deploy an Bind Grounding Resource.

https://www.microsoft.com/en-us/bing/apis/grounding-pricing?msockid=1960216b4b5f6a921268355a4acc6b0a

https://ms.portal.azure.com/#create/Microsoft.BingGroundingSearch

Option 2:
Make API call to Custom Search Bing Resource.

In [ ]:
print('hello world')

In [ ]:
from dotenv import load_dotenv 
import os # Load the .env file 
load_dotenv(override=True)

In [ ]:
print(os.environ["MODEL_DEPLOYMENT_NAME"])

## Agent with Bing Grounding

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# [START create_agent_with_bing_grounding_tool]
bing_connection = project_client.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_bing_grounding_tool]

    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What is the top news today",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

In [ ]:

data = messages.as_dict()
listmessages = data['data']  
  
for msg in listmessages:  
    content = msg['content'][0]['text']['value']  
    annotations = msg['content'][0]['text']['annotations']
    print(content)
    print(annotations)

Bing Custom Search SDK
https://github.com/Azure-Samples/cognitive-services-python-sdk-samples/tree/master?tab=readme-ov-file


In [ ]:
import os

from azure.cognitiveservices.search.customsearch import CustomSearchClient
from msrest.authentication import CognitiveServicesCredentials

load_dotenv(override=True)

In [ ]:
subscriptionKey = os.environ['BING_CUSTOM_SEARCH_SUBSCRIPTION_KEY']
endpoint = os.environ['BING_CUSTOM_SEARCH_ENDPOINT']
customConfigId = os.environ["BING_CUSTOM_CONFIG"]  # you can also use "1"

print(subscriptionKey)
print(endpoint)
print(customConfigId)

In [ ]:

# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

# You may need the below as well
# pip install pipenv
# pipenv install requests
# <importsAndVars>
import json
import os
from pprint import pprint
import requests 

'''
This sample uses the Bing Custom Search API to search for a query topic and get back user-controlled web page results.
Bing Custom Search API: https://docs.microsoft.com/en-us/bing/search-apis/bing-custom-search/overview 
'''

# Add your Bing Custom Search subscription key and endpoint to your environment variables.
subscriptionKey = os.environ['BING_CUSTOM_SEARCH_SUBSCRIPTION_KEY']
endpoint = os.environ['BING_CUSTOM_SEARCH_ENDPOINT']
customConfigId = os.environ["BING_CUSTOM_CONFIG"]  # you can also use "1"
searchTerm = "microsoft"

print(subscriptionKey)
print(endpoint)
print(customConfigId)
# </importsAndVars>
# <url>
# Add your Bing Custom Search endpoint to your environment variables.
url = endpoint + "/v7.0/custom/search?q=" + searchTerm + "&customconfig=" + customConfigId
# </url>
# <request>
r = requests.get(url, headers={'Ocp-Apim-Subscription-Key': subscriptionKey})
pprint(json.loads(r.text))
# </request>

In [ ]:




def custom_search_web_page_result_lookup():
    """CustomSearch.

    This will look up a single query (Xbox) and print out name and url for first web result.
    """

    client = CustomSearchClient(
        endpoint=url,
        credentials=CognitiveServicesCredentials(subscriptionKey))

    try:
        web_data = client.custom_instance.search(query="xbox", custom_config=1)
        print("Searched for Query 'xbox'")

        if web_data.web_pages.value:
            first_web_result = web_data.web_pages.value[0]
            print("Web Pages result count: {}".format(
                len(web_data.web_pages.value)))
            print("First Web Page name: {}".format(first_web_result.name))
            print("First Web Page url: {}".format(first_web_result.url))
        else:
            print("Didn't see any web data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))


if __name__ == "__main__":
   custom_search_web_page_result_lookup()